### Generative Adversarial Model - Group 9 <div style="text-align: right; float: right"> IANNWTF - Sheet08 </div>
In this sheet we used a generative model on the MNIST data set to generate "fake" handwritten numbers.

The training process is saved as summaries and saved in the `summary_dir` directory (it's called summary...). In tensorboard you can find the graphs for the losses, and can look at the images over the course of training. So please execute code to access graphs.

##### The usual imports

In [3]:
import numpy as np
import struct, os, time
import tensorflow as tf

/home/antonia/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


##### Lukas' helper script for loading the mnist data

In [4]:
# %load 08_mnist-gan-helper.py
class MNIST_GAN():
    def __init__(self, directory):
        self._directory = directory
        
        self._data = self._load_binaries("train-images.idx3-ubyte")
        self._data = np.append(self._data, self._load_binaries("t10k-images.idx3-ubyte"), axis = 0)
        self._data = ((self._data / 255) * 2) - 1
        self._data = self._data.reshape([-1, 28, 28, 1])

    
    def _load_binaries(self, file_name):
        path = os.path.join(self._directory, file_name)
        
        with open(path, 'rb') as fd:
            check, items_n = struct.unpack(">ii", fd.read(8))

            if "images" in file_name and check == 2051:
                height, width = struct.unpack(">II", fd.read(8))
                images = np.fromfile(fd, dtype = 'uint8')
                return np.reshape(images, (items_n, height, width))
            else:
                raise ValueError("Not a MNIST file: " + path)
    
    def get_batch(self, batch_size):
        samples_n = self._data.shape[0]
        if batch_size <= 0:
            batch_size = samples_n
            
        random_indices = np.random.choice(samples_n, samples_n, replace = False)
        data = self._data[random_indices]
        
        for i in range(samples_n // batch_size):
            on = i * batch_size
            off = on + batch_size
            yield data[on:off]


##### Lukas' layer helper file
The `back_conv_layer` was implemented by us. It is mostly analogue to the `conv_layer` function, with the biggest difference being the `target_shape` parameter, which holds the output shape to be genereated by the transposed convolution.

In [5]:
# %load 08_mnist-gan-layers.py
def feed_forward_layer(x, target_size, normalize = False, activation_function = None):
    print("Forward-Layer:" + str(x.shape))
    
    fan_in = int(x.shape[-1])
    
    if activation_function == tf.nn.relu:
        var_init = tf.random_normal_initializer(stddev = 2/fan_in)
    else:
        var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
    weights = tf.get_variable("weights", [x.shape[1], target_size], tf.float32, var_init)
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [target_size], tf.float32, var_init)
    
    activation = tf.matmul(x, weights) + biases
    
    if normalize:
        activation = batch_norm(activation, [0])
    
    return activation_function(activation) if callable(activation_function) else activation


def conv_layer(x, kernel_quantity, kernel_size, stride_size, normalize = False, activation_function = False):
    print("Conv-Layer:" + str(x.shape))
    depth = x.shape[-1]
    fan_in = int(x.shape[1] * x.shape[2])
    
    if activation_function == tf.nn.relu or activation_function == tf.nn.leaky_relu:
        var_init = tf.random_normal_initializer(stddev = 2/fan_in)
    else:
        var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
    kernels = tf.get_variable("kernels", [kernel_size, kernel_size, depth, kernel_quantity], tf.float32, var_init)
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [kernel_quantity], initializer = var_init)
    
    activation = tf.nn.conv2d(x, kernels, strides = [1, stride_size, stride_size, 1], padding = "SAME") + biases
    
    if normalize:
        activation = batch_norm(activation, [0, 1, 2])
    
    return activation_function(activation) if callable(activation_function) else activation


def back_conv_layer(x, target_shape, kernel_size, stride_size, normalize = False, activation_function = False):
    print("De-Conv-Layer:" + str(x.shape))
    depth = x.shape[-1]                   #number of activation maps in the de_conv layer (in_channels)
    fan_in = int(x.shape[1] * x.shape[2]) #resolution of the activation map, important for proper intitialization of kernel weights
    kernel_quantity = target_shape[-1]    #number of activation maps in the target layer (out_channel)
    
    if activation_function == tf.nn.relu or activation_function == tf.nn.leaky_relu:
        var_init = tf.random_normal_initializer(stddev = 2/fan_in)
    else:
        var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
        
    # switch kernel_quantity and depth (in_channels and out_channels) because we do deconvolution
    kernels = tf.get_variable("kernels", [kernel_size, kernel_size, kernel_quantity, depth], tf.float32, var_init)
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [kernel_quantity], initializer = var_init)
    
    # use conv2d_transpose as instructed
    activation = tf.nn.conv2d_transpose(x, kernels, target_shape, strides = [1, stride_size, stride_size, 1], padding = "SAME") + biases
    
    if normalize:
        activation = batch_norm(activation, [0, 1, 2])
    
    return activation_function(activation) if callable(activation_function) else activation

def flatten(x):
    size = int(np.prod(x.shape[1:]))
    return tf.reshape(x, [-1, size])

def _pop_batch_norm(x, pop_mean, pop_var, offset, scale):
    return tf.nn.batch_normalization(x, pop_mean, pop_var, offset, scale, 1e-6)

def _batch_norm(x, pop_mean, pop_var, mean, var, offset, scale):
    decay = 0.99
    
    dependency_1 = tf.assign(pop_mean, pop_mean * decay + mean * (1 - decay))
    dependency_2 = tf.assign(pop_var, pop_var * decay + var * (1 - decay))

    with tf.control_dependencies([dependency_1, dependency_2]):
        return tf.nn.batch_normalization(x, mean, var, offset, scale, 1e-6)

def batch_norm(x, axes):
    depth = x.shape[-1]
    mean, var = tf.nn.moments(x, axes = axes)
    global is_training
    
    var_init = tf.constant_initializer(0.0)
    offset = tf.get_variable("offset", [depth], tf.float32, var_init)
    var_init = tf.constant_initializer(1.0)
    scale = tf.get_variable("scale", [depth], tf.float32, var_init)
    
    pop_mean = tf.get_variable("pop_mean", [depth], initializer = tf.zeros_initializer(), trainable = False)
    pop_var = tf.get_variable("pop_var", [depth], initializer = tf.ones_initializer(), trainable = False)
    
    return tf.cond(
        is_training,
        lambda: _batch_norm(x, pop_mean, pop_var, mean, var, offset, scale),
        lambda: _pop_batch_norm(x, pop_mean, pop_var, offset, scale)
    )


##### Loading the MNIST data and hyperparameters
`batch_size` is used for the fake and the real images

In [6]:
#mnist = MNIST_GAN("")
batch_size = 32
epochs = 2
learning_rate = 0.0004
beta1 = 0.5

#gen_feature_maps = [64, 32, 16] # target shapes for generator layers

# for tensorboard
weight_dir = "weights/"
summary_dir = "summary/"

## Hyperparameters
embedding_size = 64
lstm_memory_size = 64
ingredient_amount = 20000 # however many there are
cutoff_length = 300
subsequence_length = 300
batch_size = 250
epochs = 6
learning_rate = 0.05
dropout_rate = 0.85
optimizer = tf.train.AdamOptimizer
layer1_size = 512
layer2_size = 64
layer_out_size = 1


z_size = 50 # length of random input vector
max_ingredients = 15

starter_learning_rate = 0.05

dis_feature_maps = [ 8, 16, 32] # target shapes for discriminator layers. maybe to be changed? dunno

In [28]:
def get_ings(embeddings, ingredients):
    '''
    embeddings: the weight matrix with all ingredients representations shape: [ingredient_amount, lstm_memory_size]
    ingredients: a matrix of ingedient vectors of who we want to find the nearest neigbhours
      
    '''''
    normed_embedding = tf.nn.l2_normalize(embeddings, dim=1)
    if (type(ingredients)!= list):
        arrays = ingredients
    #else: # we shouldn't have IDs, delete later
     #   ingredients = np.array(ingredients)
      #  arrays = tf.nn.embedding_lookup(embeddings, ingredients)
    
    # initalize "recipes". not sure yet how or if the loop thing is even the optimal way to do it
    
    recipes = []
    
    arraylist = tf.unstack(arrays)
    
    for array in arraylist : # get each recipe (sample in batch_size)
        
        normed_array = tf.nn.l2_normalize(array, dim=1) # what is this for?

        cosine_similarity = tf.matmul(normed_array, tf.transpose(normed_embedding, [1, 0]))
        
        closest_ings = tf.argmax(cosine_similarity, 1)  # shape [max_ingredients], type int64. we only need 1 ingredient, so no need for top_k anymore
        
        # stack this on top of recipes so we get batch_size, max_ingredients as output size in the end.
        # todo: actually implement.
        recipes = recipes + [closest_ings]
    
    recipes = tf.stack(recipes)
    return recipes # returns index of closest ingredient. let's hope the dimension stuff works out in this function

In [30]:
# Defining the graph
tf.reset_default_graph()
   
###### The generator network. ######      
with tf.variable_scope("generator"):
  
    # see homework for original gen network

    ## placeholders
    with tf.variable_scope("inputs"):
        input_vec = tf.placeholder(tf.float32,[batch_size,z_size]) # for random input
        is_training = tf.placeholder(tf.bool, [])
        
        # what do the dimensions of this denote again?
        # i put *max_ingredients so it doesn't crash. like where I created the BasicLSTMCell a couple of lines later. how many "units" does it need?
        cell_state = tf.placeholder(tf.float32,[batch_size, lstm_memory_size*max_ingredients],name="cell_state")
        hidden_state = tf.placeholder(tf.float32,[batch_size, lstm_memory_size*max_ingredients],name="hidden_state")
        
        # original recipe placeholder. should be batch_size, max_ingredients, 3 later but for now we'll only take ingreidnets
        #orig_recipes = batch_norm(tf.placeholder(tf.int64, [batch_size,max_ingredients]), [0,1,2])

        orig_recipes = batch_norm(tf.placeholder(tf.int64, [batch_size,max_ingredients]), [0,1])


    ##### rnn
    with tf.variable_scope("rnn"):
        cell = tf.nn.rnn_cell.BasicLSTMCell(lstm_memory_size*max_ingredients)
        cell= tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=dropout_rate)
        zero_state = cell.zero_state(batch_size, tf.float32)
        state = tf.nn.rnn_cell.LSTMStateTuple(c = cell_state, h = hidden_state) # should be fine so far? no changes needed?
        
        outputs, state = tf.nn.static_rnn(cell, [input_vec], initial_state = state)
        
        # so at least in the example, lstm_memory_size == embedding_size.
        # so what do we do about the subsequence length?
        # for now I'll just ignore since we haven't even used sequences but the input_vec only. veeeery unsure about
        # this though.
        # so here I will state what the output should look like but not sure how we get the LSTM to produce this exact amount of data
        outputs = tf.reshape(tf.concat(outputs, axis=1), [batch_size, max_ingredients, lstm_memory_size])


    ##### embedding
    with tf.variable_scope("embedding"):
        init = tf.random_uniform_initializer(-1.0,1.0)
        embeddings = tf.get_variable("embedding", [ingredient_amount, embedding_size], initializer=init)


        ##fake_recipes will have dimension batch_size*max_ingredients*3 just like the original ones (for now without 3)             
        # so get_ings will have to return batch_size*max_ingredients for now.
        # might or might not work at the current state :D
        fake_recipes = get_ings(embeddings,outputs) 
        
        
    train_gen = tf.get_variable_scope().get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)


# create input batch for discriminator
batch = tf.concat([fake_recipes, orig_recipes],axis=0)

##### The discriminator network #####
# does this need to be adapted according to new data shape?
# i mean, we probably cannot convolve it this easily right? does convolution even make that much sense here?
train_dis = []
with tf.variable_scope("discriminator"):
    with tf.variable_scope("layer1"):
        fm = dis_feature_maps[0]
        conv1 = conv_layer(batch,fm,5,2,True,tf.nn.leaky_relu)
    with tf.variable_scope("layer2"):
        fm = dis_feature_maps[1]
        conv2 = conv_layer(conv1,fm,5,2,True,tf.nn.leaky_relu)
    with tf.variable_scope("layer3"):
        fm = dis_feature_maps[2]
        conv3 = conv_layer(conv2,fm,5,2,True,tf.nn.leaky_relu)
    with tf.variable_scope("readout"):
        conv3 = tf.reshape(conv3,[batch_size*2, -1])
        logits = feed_forward_layer(conv3,1,False,None)
    
    # get trainable variables for discriminator
    train_dis = tf.get_variable_scope().get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

##### Evaluation and training of the network #####
with tf.variable_scope("evaluation"):
    
    # to compute cross entropy for generator. consists of only "1" because this is the discriminator label for "real"
    gen_labels = tf.ones((batch_size,1))
    
    # only the first half of the output from the discriminator concerns the pictures produced by the generator
    # so only get first half of logits (which equals the batch_size)
    gen_logits = logits[:batch_size]
    
    # for discriminator cross entropy. first half of input are fake images ("0"), second half real ones ("1")
    dis_labels = tf.concat((tf.zeros((batch_size,1)),tf.ones((batch_size,1))),axis=0)
    dis_logits = logits
    
    gen_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=gen_labels, logits=gen_logits)
    dis_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=dis_labels, logits=dis_logits)
    
    # tensorboard stuff
    tf.summary.scalar("generator_loss", tf.reduce_mean(gen_loss))
    tf.summary.scalar("discriminator_loss", tf.reduce_mean(dis_loss))
    
    global_step = tf.get_variable("global_step",[],tf.int32,trainable=False)
    
    # initialize optimizer
    optimizer  = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)
    
    # define training steps with respective variable lists
    gen_step = optimizer.minimize(gen_loss, var_list=train_gen, global_step=global_step)
    dis_step = optimizer.minimize(dis_loss, var_list=train_dis, global_step=global_step)
    
    summaries = tf.summary.merge_all()

TypeError: unsupported operand type(s) for *: 'Tensor' and 'float'

##### This is where we train our network

In [6]:
# HAVE ONLY MADE SMALL ADJUSTMENTS. don't think there's a point if we don't have data


with tf.Session() as session:
    ## initialising directories if they do not exist
    if not os.path.exists(weight_dir):
        os.makedirs(weight_dir)
    if not os.path.exists(summary_dir):
        os.makedirs(summary_dir)
    train_saver = tf.train.Saver()
    train_writer = tf.summary.FileWriter(summary_dir, session.graph)
    
    ## Try to load trained data and initialise variables if there is none
    ckpt = tf.train.latest_checkpoint(weight_dir)
    if ckpt is None:
        print("No checkpoint found. Initialising variables.")
        session.run(tf.global_variables_initializer())
    else:
        train_saver.restore(session, ckpt)
    
    ## Train all the epochs and save the summaries for each timestep
    for epoch in range(1, epochs+1):
        
        print("Starting epoch %d..." %epoch, end='')
        
        real_data = # get batch of original recipes #mnist.get_batch(batch_size)
        t = time.time()
        
        for data in real_data:
            ## for each batch we need a new set of random vectors
            z_vec = np.random.uniform(-1, 1, (batch_size,z_size))
            
            # feed data into placeholders
            feed_dict = {input_vec: z_vec, orig_recipes: data, is_training: True}
            
            # let's train our network!
            _gStep, _dStep, _summ, _step = session.run([gen_step, dis_step, summaries, global_step],
                                                         feed_dict=feed_dict)
            train_writer.add_summary(_summ, _step)
            
        t = time.time() - t
        est = t * (epochs - epoch)
        
        ## This is just the time formatting. Move along.
        m, s = divmod(t, 60)
        e_m, e_s = divmod(est, 60)
        
        print(("Finished epoch in {0: .0f}m{1: .2f}s. "+\
              "Estimated remaining time: {2: .0f}m{3: .2f}s").format(m,s,e_m,e_s))
        
        train_saver.save(session, weight_dir, global_step=_step)

No checkpoint found. Initialising variables.
Starting epoch 1...Finished epoch in  2m 30.52s. Estimated remaining time:  2m 30.52s
Starting epoch 2...Finished epoch in  2m 39.61s. Estimated remaining time:  0m 0.00s
